<a href="https://colab.research.google.com/github/Aftabgazali/Custom_DataSet_Using_Pytorch/blob/main/Custom_DataSet_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# Setting up Device Agnostic Code

In [ ]:
device ="cuda" if torch.cuda.is_available() else "cpu"
device

# Getting a Subset of Food101 dataset


In [ ]:
import requests
import zipfile
from pathlib import Path

# setup the path to data folder
data_path = Path("data/")
image_path = data_path /"pizza_steak_sushi"

# If the image folder exist, download it and prepare....
if image_path.is_dir():
  print(f"{image_path} already exist")
else:
  print(f"{image_path} doesn't exist creating one...")
  image_path.mkdir(parents=True, exist_ok= True)

# Dowload the data, open the file as write binary, make the request (click on raw and copy link)
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading Pizza Steak & Sushi data")
  # write the contents inside the zip file
  f.write(request.content)

# Unzip the data file and read the data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip" ,"r") as zip_ref:
  print("Unzipping Pizza Steak & Sushi data....")
  zip_ref.extractall(image_path)

# Preparing the Data and Visualizing it

In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walksthrough the dir_path returning it's contents

  """
  # Walk into each and every directories of pizza_steak_sushi
  for dirpath, dirnames, filenames in os.walk(dir_path):
    # print(f"This is a dirname {dirnames} & this is the filenames {filenames}")
    print(f"There are {len(dirnames)} directories & {len(filenames)} images in '{dirpath}'. ")

In [ ]:
walk_through_dir(image_path)

## Setting up the train & test dir for our ImageFolder

In [ ]:
# Setup train & test paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

In [ ]:
import random
from PIL import Image

# random.seed(42)

# Get ALl the Image paths and glob them together in a list that matches a pattern: train/ any folder either sushi,steak, pizza/ pick anything that ends with .jpg
image_path_lists = list(image_path.glob("train/*/*.jpg"))

# Pick a random Image from the given list
random_image = random.choice(image_path_lists)

random_image

# Get the Image class from path name eg: data/pizza_steak_sushi/train/sushi/385154.jpg, the image class is sushi
image_class = random_image.parent.stem
image_class

# Open the Image
img = Image.open(random_image)

# Print the metadata
print(f"Random image path: {random_image}")
print(f"Image class: {image_class}")
print(f"Image Height: {img.height} width: {img.width} Size: {img.size}")
img

## Visualizing using matplotlib

In [ ]:
img_as_array = np.asarray(img)
plt.figure(figsize=(10,7))
plt.imshow(img_as_array)
plt.title(f"Image Class: {image_class} | Image Shape: {img_as_array.shape} ")

In [ ]:
# Image is in Numerical format
img_as_array[0]

## One Conclusion is that the Images are not in same size Heigth is different for diff. Images

# Preparing Data in Pytorch Tensors

Before we can use our Image data with Pytorch:

1. Turn your target data into tensors

2. Convert the data into a `torch.utils.data.Dataset` and eventually a `torch.utils.data.DataLoader`, `Datasets` & `Batches` inorder for the model to train it easily(Faster..)


##  Transforming the Data with `torchvision.transforms`


In [ ]:
transform = transforms.Compose([
    # Resize our Images to 64, 64
    transforms.Resize(size=(64,64)),
    # Randomly flip the image with 50% prob.
    transforms.RandomHorizontalFlip(p=0.5),
    # Turn the image into torch.tensor: Converts a PIL Image or ndarray into Tensor from a range[0,255] to [0.0,1.0]
    transforms.ToTensor()
    ])

In [ ]:
# [0,255] --> [0.0, 1.0]
transform(img)

## Visualizing the Transformed Data

In [ ]:
def plot_transformed_image(image_path, transform, n=3):
  # Select 3 random images from the image path list
  random_image_paths = random.sample(image_path, k=n)

  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(nrows=1,ncols=2)
      ax[0].imshow(f)
      ax[0].set_title(f"Original\nSize: {f.size}")
      ax[0].axis(False)
      # Apply the transformation
      transformed_img = transform(f).permute(1,2,0) # note: we are going from [C,H,W] ---> [H,W,C] why permute? becuase matplot lib accepts image in the form [H,W,C] so we move H at first pos, W at 2nd Pos, C at 3rd Pos.
      ax[1].imshow(transformed_img)
      ax[1].set_title(f"Transformed Image\nSize: {transformed_img.shape}")
      ax[1].axis(False)

      fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=12)

In [ ]:
plot_transformed_image(image_path=image_path_lists, transform=transform)

## 1. Loading the Data using ImageFolder

In [ ]:
train_data = ImageFolder(root=train_dir, # From Training Directory Image_path/train: The ImageFolder accepts data in the form `/train/images.jpg & `/test/images.jpg
                         transform=transform)

test_data = ImageFolder(root=test_dir,
                        transform=transform)

train_data, test_data

In [ ]:
# GEt Class Names
class_names = train_data.classes
class_names

In [ ]:
# Get Class with Dict
class_idx = train_data.class_to_idx
class_idx

In [ ]:
# Check the length of the dataset
len(train_data), len(test_data)

In [ ]:
image, label = train_data[0]


class_names[label]

In [ ]:
# Reorder the Image Dimensions
img_permute = image.permute(1,2,0)
print(f"Original Shape: {image.shape} ")
print(f"Image permute Shape: {img_permute.shape}")

# Plot the Image
plt.figure(figsize=(10,7))
plt.imshow(img_permute)
plt.title(class_names[label])
plt.axis(False)

# Preparing the Data into Batches

In [ ]:
BATCH_SIZE = 1
train_data_loader = DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True)
test_data_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
len(train_data_loader), len(test_data_loader)

In [ ]:
img, label = next(iter(train_data_loader))

print(f"Image shape: {img.shape}, Label {label.shape}")

# Other forms of Transformation: Data Augmentation

*Process of artificially Increase/adding diversity to your training data.*
* It helps model, to pick out patterns from a diverese pool of images

### Let's look at the Trivial Augmentation

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    # Apply Trivial Augment
    # num_bins: How Instense you want the transformation to be 31 is max.
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

In [ ]:
image_path_lists = list(image_path.glob("train/*/*.jpg"))
image_path_lists[:5]

In [ ]:
plot_transformed_image(image_path=image_path_lists,n=3, transform=train_transform)

# Model_V0: TinyVGG without data augmentation
Let's replicate the TinyVGG Model

## Creating transforms and loading data for Model 0

In [ ]:
simple_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
train_data_for_model_v0 = ImageFolder(root=train_dir,transform=simple_transform,target_transform=None)
test_data_for_model_v0 = ImageFolder(root=test_dir,transform=simple_transform,target_transform=None)

In [ ]:
# Turning into Batches of Images
BATCH_SIZE = 32
train_data_loader_model_v0 = DataLoader(dataset=train_data_for_model_v0, batch_size=BATCH_SIZE, shuffle=True)
test_data_loader_model_v0 = DataLoader(dataset=test_data_for_model_v0, batch_size=BATCH_SIZE)

In [ ]:
len(train_data_loader_model_v0), len(test_data_loader_model_v0)

## Building the TinyVGG Model

In [ ]:
class TinyVGGModel_V0(nn.Module):
  """
  Model architecture copying TinyVGG Model
  """
  def __init__(self, input_shape: int, output_shape: int, hidden_units:int):
    super().__init__()

    self.conv_layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.conv_layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features = output_shape)
    )

  # Forward Pass
  def forward(self, x:torch.Tensor):
    #print(x.shape)
    x = self.conv_layer_1(x)
    #print(x.shape)
    x = self.conv_layer_2(x)
    #print(x.shape)
    return self.classifier_layer(x)

# Creating a Instance of the TinyVGG Model
model_v0 = TinyVGGModel_V0(input_shape=3, output_shape= len(class_names), hidden_units=10)

In [ ]:
# Create a Random Image
random_image = torch.randn(size=(3,64,64))
predicted_image = model_v0(random_image.unsqueeze(0))
# We got the value which is hidden_units*16*16

In [ ]:
# Similarly we can do:
rand_image, rand_label = next(iter(train_data_loader_model_v0))
predicted_image = model_v0(rand_image)

## Printing the Summary for Model_V0
Use `torchinfo` to summarize your model  

In [ ]:
# Install torchinfo
try:
  import torchinfo
except:
  !pip install torchinfo

from torchinfo import summary
# Give a Single batch of image, you can try with 32
summary(model_v0, input_size=[1,3,64,64])
# Output of last MaxPool Layer will be used to find the metrics to multiply in the Classifier Layer

## Setting up Model Loss & Optimizer

In [ ]:
model_loss = nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(params=model_v0.parameters(), lr=0.001)

In [ ]:
def model_accuracy(y_true, y_pred):
  acc = torch.eq(y_pred, y_true).sum().item()
  return (acc / len(y_true))*100

## Setting up Training & Testing Step

In [ ]:
from tqdm.auto import tqdm
def train_step(model:torch.nn.Module, data_loader: torch.utils.data, model_loss: torch.nn.Module, model_optimizer: torch.optim, model_acc,device:torch.device=device):
  # Training Mode:
  model.train()
  train_loss, train_acc = 0,0
  for batch, (X,y) in enumerate(data_loader):
    # Put X & y to the device (cpu/gpu)
    X,y = X.to(device), y.to(device)

    # Forward Pass
    y_logits = model(X)

    # Calculate the Training Loss
    loss = model_loss(y_logits, y)
    train_loss+=loss

    # Calculate the Training Accuracy
    train_acc+=model_acc(y,y_logits.argmax(dim=1))

    # Optimizer Zero grad
    model_optimizer.zero_grad()

    # Loss Backwards
    loss.backward()

    # Optimizer Step
    model_optimizer.step()

    # if batch % 2 == 0:
    #   print(f"Looked through {batch * len(X)} / {len(data_loader.dataset)}")

  # Update the training & accuracy & loss
  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  return train_loss, train_acc
  #print(f"Training Loss {train_loss:.2f} | Training Accuracy {train_acc:.2f}")

In [ ]:
#train_step(model=model_v0,no_of_epochs=5,data_loader=train_data_loader_model_v0, model_loss=model_loss, model_optimizer=model_optimizer,model_acc=model_accuracy)

In [ ]:
from tqdm.auto import tqdm
def test_step(model:torch.nn.Module, data_loader: torch.utils.data, model_loss: torch.nn.Module, model_acc,device:torch.device=device):
    # Testing Mode:
    model.eval()
    with torch.inference_mode():
      test_loss, test_acc = 0,0
      for (X_test,y_test) in (data_loader):
        # Put X & y to the device (cpu/gpu)
        X_test,y_test = X_test.to(device), y_test.to(device)

        # Forward Pass
        y_logits = model(X_test)

        # Calculate the Training Loss
        loss = model_loss(y_logits, y_test)
        test_loss+=loss

        # Calculate the Training Accuracy
        test_acc+=model_acc(y_test,y_logits.argmax(dim=1))

      # Update the training & accuracy & loss
      test_loss /= len(data_loader)
      test_acc /= len(data_loader)

      return test_loss, test_acc
      #print(f"Testing Loss {test_loss:.2f} | Testing Accuracy {test_acc:.2f}")

In [ ]:
#test_step(model=model_v0,data_loader=test_data_loader_model_v0, model_loss=model_loss,model_acc=model_accuracy)

In [ ]:
def train_model(model:torch.nn.Module, no_of_epochs:int, train_dataloader: torch.utils.data, test_dataloader:torch.utils.data, model_loss:torch.nn.Module,model_optimizer:torch.optim,model_acc,device:torch.device=device):
  """
  Applies both Training & Testing Step for a Model, combines both the functions
  """
  results = {
      "train_loss":[],
      "train_acc":[],
      "test_loss":[],
      "test_acc":[]
  }
  for epoch in tqdm(range(no_of_epochs)):
    train_loss, train_acc = train_step(model=model,data_loader=train_dataloader, model_loss=model_loss, model_optimizer=model_optimizer,model_acc=model_acc)
    test_loss, test_acc = test_step(model=model,data_loader=test_dataloader, model_loss=model_loss,model_acc=model_acc)
    results['train_loss'].append(train_loss.item())
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss.item())
    results['test_acc'].append(test_acc)
    print(f"Epoch: {epoch} Training Loss {train_loss:.2f} | Training Accuracy {train_acc:.2f} | Testing Loss {test_loss:.2f} | Testing Accuracy {test_acc:.2f}")

  return {'model':model.__class__.__name__, 'Loss': f'{test_loss.item():.2f}','Accuracy': f'{test_acc:.2f}'}, results

In [ ]:
model_v0_results, model_v0_loss_acc_results_per_epoch = train_model(model_v0,5, train_data_loader_model_v0, test_data_loader_model_v0,model_loss,model_optimizer,model_accuracy)

In [ ]:
model_v0_results

In [ ]:
model_v0_loss_acc_results_per_epoch

## Printing out the Loss & Accuracy curves for the Model

In [ ]:
model_v0_loss_acc_results_per_epoch.keys()

In [ ]:
def print_loss_curve(model_v0_loss_acc_results_per_epoch):
    # Plotting the loss curves
    train_loss = model_v0_loss_acc_results_per_epoch["train_loss"]
    test_loss = model_v0_loss_acc_results_per_epoch["test_loss"]
    epochs = range(len(train_loss))

    sns.set(style="whitegrid")  # Set Seaborn style

    plt.figure(figsize=(8, 5))  # Set figure size

    # Plot train loss
    sns.lineplot(x=epochs, y=train_loss, marker='o', color='b', label='Train Loss')

    # Plot test loss
    sns.lineplot(x=epochs, y=test_loss, marker='o', color='g', label='Test Loss')

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Loss Curve between Train and Test")
    plt.show()

# Example usage
# Assuming model_v0_loss_acc_results_per_epoch is a dictionary containing "train_loss" and "test_loss" lists
# model_v0_loss_acc_results_per_epoch = {"train_loss": [0.5, 0.4, 0.3], "test_loss": [0.6, 0.5, 0.4]}
# print_loss_curve(model_v0_loss_acc_results_per_epoch)


In [ ]:
print_loss_curve(model_v0_loss_acc_results_per_epoch)

### **On Visualizing we can deduce that, our model is overfitting as the train loss curve is going down, however, the test loss curve isn't that means it is performing pretty well with train data but isn't with test data**

***One of the ways we can avoid overfitting, is to get more data, apply more augmentation, Better data, Transfer Learning, the Learning rate decay https://pytorch.org/docs/stable/optim.html, simplify your model, Loss Curve Early Stopping(Stop at the epoch when the model loss was best)***



***One of the ways we can avoid underfitting, is to add more layer to your model, Tweak the Learning rate, Train for Longer, use Transfer Learning***


## Priniting out the Accuracy curves

In [ ]:
def print_acc_curve(model_v0_loss_acc_results_per_epoch):
    # Plotting the loss curves
    train_acc = model_v0_loss_acc_results_per_epoch["train_acc"]
    test_acc = model_v0_loss_acc_results_per_epoch["test_acc"]
    epochs = range(len(train_acc))

    sns.set(style="whitegrid")  # Set Seaborn style

    plt.figure(figsize=(8, 5))  # Set figure size

    # Plot train loss
    sns.lineplot(x=epochs, y=train_acc, marker='o', color='b', label='Train Acc')

    # Plot test loss
    sns.lineplot(x=epochs, y=test_acc, marker='o', color='g', label='Test Acc')

    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.title("Accuracy Curve between Train and Test")
    plt.show()

In [ ]:
print_acc_curve(model_v0_loss_acc_results_per_epoch)

# Model_V1: TinyVGG with Data Augmentation

In [ ]:
train_transform_trivial = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
test_transform_trivial = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [ ]:
train_data_augmented = ImageFolder(root=train_dir, transform=train_transform_trivial)
test_data_augmented = ImageFolder(root=test_dir, transform=test_transform_trivial)

In [ ]:
# Turning data into Batches of size 32
BATCH_SIZE = 32
train_data_loader_model_v1 = DataLoader(dataset=train_data_augmented, batch_size=BATCH_SIZE,shuffle=True)
test_data_loader_model_v1 = DataLoader(dataset=test_data_augmented, batch_size=BATCH_SIZE,shuffle=False)

In [ ]:
len(train_data_loader_model_v1), len(test_data_loader_model_v1)

## Building TinyVGG Model

In [ ]:
class TinyVGGModel_V1(nn.Module):
  """
  Model architecture copying TinyVGG Model
  """
  def __init__(self, input_shape: int, output_shape: int, hidden_units:int):
    super().__init__()

    self.conv_layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.conv_layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.classifier_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features = output_shape)
    )

  # Forward Pass
  def forward(self, x:torch.Tensor):
    #print(x.shape)
    x = self.conv_layer_1(x)
    #print(x.shape)
    x = self.conv_layer_2(x)
    #print(x.shape)
    return self.classifier_layer(x)

# Creating a Instance of the TinyVGG Model
model_v1 = TinyVGGModel_V1(input_shape=3, output_shape= len(class_names), hidden_units=10)

In [ ]:
model_loss = nn.CrossEntropyLoss()
model_optimizer = torch.optim.Adam(params=model_v1.parameters(), lr= 0.001)

In [ ]:
model_v1_results, model_v1_loss_acc_results_per_epoch = train_model(model_v1,5, train_data_loader_model_v1, test_data_loader_model_v1,model_loss,model_optimizer,model_accuracy)

## Plotting the loss curves for Model

In [ ]:
print_loss_curve(model_v1_loss_acc_results_per_epoch)

## Made Improvements with the Loss Curve, so it's much better than before. However, we are now in underfitting

## Printing Accuracy Curve for our Model

In [ ]:
print_acc_curve(model_v1_loss_acc_results_per_epoch)